In [3]:
!pip install scikit-learn==1.0.0
!pip install wget==3.2
!pip install gensim==3.6.0
!pip install psutil==5.4.8
!pip install spacy==2.2.4


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.corpus import state_union
from nltk.tag import StanfordPOSTagger
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from keras.layers import Dense, Input, Embedding, Lambda, Layer, Multiply, Dropout, Dot

import gensim
from gensim import utils
import sys
from sklearn.datasets import fetch_20newsgroups
from nltk import word_tokenize
from nltk import download
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
%matplotlib inline


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
stop_words = set(stopwords.words('english'))

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
train_file_path = '/content/drive/MyDrive/Event_Extraction/files/train_attn_sp.txt'
val_file_path = '/content/drive/MyDrive/Event_Extraction/files/val_attn_sp.txt'
test_file_path = '/content/drive/MyDrive/Event_Extraction/files/test_attn_sp.txt'


In [8]:
import os
import wget
import gzip
import shutil

In [9]:
gn_vec_path = "/content/drive/MyDrive/Event_Extraction/embeddings/GoogleNews-vectors-negative300.bin"
if not os.path.exists("/content/drive/MyDrive/Event_Extraction/embeddings/GoogleNews-vectors-negative300.bin"):
    if not os.path.exists("/content/drive/MyDrive/Event_Extraction/embeddings/GoogleNews-vectors-negative300.bin"):
        #Downloading the reqired model
        if not os.path.exists("/content/drive/MyDrive/Event_Extraction/embeddings/GoogleNews-vectors-negative300.bin.gz"):
            if not os.path.exists("/content/drive/MyDrive/Event_Extraction/embeddings/GoogleNews-vectors-negative300.bin.gz"):
                wget.download("https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz")
            gn_vec_zip_path = "/content/drive/MyDrive/Event_Extraction/embeddings/GoogleNews-vectors-negative300.bin.gz"
        else:
            gn_vec_zip_path = "/content/drive/MyDrive/Event_Extraction/embeddings/GoogleNews-vectors-negative300.bin.gz"
        #Extracting the required model
        with gzip.open(gn_vec_zip_path, 'rb') as f_in:
            with open(gn_vec_path, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
    else:
        gn_vec_path = "/content/drive/MyDrive/Event_Extraction/embeddings/GoogleNews-vectors-negative300.bin" + gn_vec_path

print(f"Model at {gn_vec_path}")

Model at /content/drive/MyDrive/Event_Extraction/embeddings/GoogleNews-vectors-negative300.bin


In [10]:
import warnings #This module ignores the various types of warnings generated
warnings.filterwarnings("ignore") 

import psutil #This module helps in retrieving information on running processes and system resource utilization
process = psutil.Process(os.getpid())
from psutil import virtual_memory
mem = virtual_memory()

import time #This module is used to calculate the time  

In [11]:
from gensim.models import Word2Vec, KeyedVectors
pretrainedpath = gn_vec_path

#Load W2V model. This will take some time, but it is a one time effort! 
pre = process.memory_info().rss
print("Memory used in GB before Loading the Model: %0.2f"%float(pre/(10**9))) #Check memory usage before loading the model
print('-'*10)

start_time = time.time() #Start the timer
ttl = mem.total #Toal memory available

model = KeyedVectors.load_word2vec_format(pretrainedpath, binary=True) #load the model
print("%0.2f seconds taken to load"%float(time.time() - start_time)) #Calculate the total time elapsed since starting the timer
print('-'*10)

print('Finished loading Word2Vec')
print('-'*10)

post = process.memory_info().rss
print("Memory used in GB after Loading the Model: {:.2f}".format(float(post/(10**9)))) #Calculate the memory used after loading the model
print('-'*10)

print("Percentage increase in memory usage: {:.2f}% ".format(float((post/pre)*100))) #Percentage increase in memory after loading the model
print('-'*10)

print("Number of words in vocablulary: ",len(model.vocab)) #Number of words in the vocabulary. 

Memory used in GB before Loading the Model: 0.58
----------
67.98 seconds taken to load
----------
Finished loading Word2Vec
----------
Memory used in GB after Loading the Model: 5.44
----------
Percentage increase in memory usage: 936.75% 
----------
Number of words in vocablulary:  3000000


In [12]:
my_trained_vectors = model.wv
# save vectors to file to use them later
my_trained_vectors.save_word2vec_format('/content/drive/MyDrive/Event_Extraction/embeddings/GoogleNews-vectors-negative300.txt', fvocab='GoogleNews-vectors-negative300_vocab.txt', binary=False)
#my_trained_vectors.save_word2vec_format('/content/drive/MyDrive/Event_Extraction/embeddings/embeddings.txt', binary=False)

In [13]:
vec = model['man']
print(vec)

[ 0.32617188  0.13085938  0.03466797 -0.08300781  0.08984375 -0.04125977
 -0.19824219  0.00689697  0.14355469  0.0019455   0.02880859 -0.25
 -0.08398438 -0.15136719 -0.10205078  0.04077148 -0.09765625  0.05932617
  0.02978516 -0.10058594 -0.13085938  0.001297    0.02612305 -0.27148438
  0.06396484 -0.19140625 -0.078125    0.25976562  0.375      -0.04541016
  0.16210938  0.13671875 -0.06396484 -0.02062988 -0.09667969  0.25390625
  0.24804688 -0.12695312  0.07177734  0.3203125   0.03149414 -0.03857422
  0.21191406 -0.00811768  0.22265625 -0.13476562 -0.07617188  0.01049805
 -0.05175781  0.03808594 -0.13378906  0.125       0.0559082  -0.18261719
  0.08154297 -0.08447266 -0.07763672 -0.04345703  0.08105469 -0.01092529
  0.17480469  0.30664062 -0.04321289 -0.01416016  0.09082031 -0.00927734
 -0.03442383 -0.11523438  0.12451172 -0.0246582   0.08544922  0.14355469
 -0.27734375  0.03662109 -0.11035156  0.13085938 -0.01721191 -0.08056641
 -0.00708008 -0.02954102  0.30078125 -0.09033203  0.03149

most_similar allows for vector arithmetic (as the average value of the input positive/negative vectors, where negative vectors are first multiplied by -1). 

Two things to note while using pre-trained models:
Tokens/Words are always lowercased. If a word is not in the vocabulary, the model throws an exception.
So, it is always a good idea to encapsulate those statements in try/except blocks.

In [14]:
!python -m spacy download en_core_web_md

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 96.4 MB 1.4 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [15]:
import spacy

%time 
nlp = spacy.load('en_core_web_md')
# process a sentence using the model
mydoc = nlp("Canada is a large country")
#Get a vector for individual words
#print(doc[0].vector) #vector for 'Canada', the first word in the text 
print(mydoc.vector) #Averaged vector for the entire sentence

CPU times: user 9 µs, sys: 0 ns, total: 9 µs
Wall time: 17.2 µs
[-1.12055197e-01  2.26087615e-01 -5.15111461e-02 -1.21812008e-01
  4.13958639e-01 -8.56475979e-02 -2.84600933e-03 -2.26096585e-01
  6.98113963e-02  2.27946019e+00 -4.49774921e-01 -6.39050007e-02
 -1.80326015e-01 -8.79765972e-02  9.93399299e-04 -1.57384202e-01
 -1.23817801e-01  1.54990411e+00  2.00794004e-02  1.38399601e-01
 -1.48897991e-01 -2.23025799e-01 -1.48171991e-01  4.68924567e-02
 -3.17026004e-02  1.19096041e-02 -6.10985979e-02  9.57068056e-02
  9.37099904e-02  1.70955807e-01 -9.29740071e-03  7.88536817e-02
  1.74508005e-01 -1.04450598e-01  1.04872189e-01 -1.16961405e-01
  6.23028055e-02 -2.23016590e-01 -1.44107476e-01 -2.03423887e-01
  2.61404991e-01  2.43404001e-01  1.51980996e-01 -1.12484001e-01
  1.18055798e-01 -9.51323956e-02  8.66319984e-02 -2.54322797e-01
  3.84932049e-02  1.18278004e-01 -3.21602583e-01  3.73764008e-01
  1.13018408e-01 -8.05834010e-02  1.84921592e-01  9.38879885e-03
  1.22166201e-01 -3.242880

The archive contains text-encoded vectors of various sizes: 50-dimensional,
100-dimensional, 200-dimensional, 300-dimensional. We'll use the 100D ones.

Let's make a dict mapping words (strings) to their NumPy vector representation:

In [16]:
with open(train_file_path) as f:
    train_samples = f.readlines()

#train_samples = pd.read_csv('/content/drive/MyDrive/Event_Extraction/files/train_attn_sp.txt')
from tensorflow.keras.layers import TextVectorization


vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)
vectorizer.adapt(text_ds)

In [17]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [18]:
path_embedding = os.path.join(
    os.path.expanduser("~"), "/content/drive/MyDrive/Event_Extraction/embeddings/embeddings.txt"
)

embeddings_index = {}
with open(path_embedding) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 783879 word vectors.


Now, let's prepare a corresponding embedding matrix that we can use in a Keras
`Embedding` layer. It's a simple NumPy matrix where entry at index `i` is the pre-trained
vector for the word of index `i` in our `vectorizer`'s vocabulary.

In [19]:
num_tokens = len(voc) + 2
embedding_dim = 300
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))


Converted 15360 words (4640 misses)


Next, we load the pre-trained word embeddings matrix into an `Embedding` layer.

Note that we set `trainable=False` so as to keep the embeddings fixed (we don't want to
update them during training).

In [20]:
from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)